In [ ]:
# Анализ новостного потока как влияние на технический анализ
# Кафедра ИИ, магистерская работа

In [ ]:
Импорт необходимых библиотек

import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

sns.set(style='whitegrid')

In [ ]:
1. Загрузка данных по акциям AAPL

ticker = 'AAPL'
start_date = '2022-01-01'
end_date = '2023-12-31'

df = yf.download(ticker, start=start_date, end=end_date)
df.reset_index(inplace=True)

# Добавим технические индикаторы: RSI, MACD
delta = df['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))

exp1 = df['Close'].ewm(span=12, adjust=False).mean()
exp2 = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = exp1 - exp2
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

In [ ]:
2. Загрузка и подготовка новостного потока

news_df = pd.read_csv('AAPL_news_extended.csv')
news_df['date'] = pd.to_datetime(news_df['date'])
df['Date'] = pd.to_datetime(df['Date'])

# Объединение по дате
merged_df = pd.merge(df, news_df, how='left', left_on='Date', right_on='date')
merged_df.drop(columns=['date'], inplace=True)

In [ ]:
3. Анализ тональности новостей с помощью модели BERT

sentiment_analyzer = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

# Применяем к небольшому числу заголовков (примерно 100 для демонстрации)
sample_news = merged_df['headline'].dropna().unique()[:100]
sentiment_results = sentiment_analyzer(list(sample_news))

sentiment_map = []
for r in sentiment_results:
    label = r['label']
    if '1' in label or '2' in label:
        sentiment_map.append('negative')
    elif '3' in label:
        sentiment_map.append('neutral')
    else:
        sentiment_map.append('positive')

news_sentiment_df = pd.DataFrame({
    'headline': sample_news,
    'bert_sentiment': sentiment_map
})

# Добавим оценки обратно
merged_df = pd.merge(merged_df, news_sentiment_df, on='headline', how='left')

In [ ]:
4. Визуализация зависимости цен от тональности новостей

colors = {'positive': 'green', 'neutral': 'gray', 'negative': 'red'}

plt.figure(figsize=(14, 6))
plt.plot(merged_df['Date'], merged_df['Close'], label='Close Price', color='black')

for idx, row in merged_df.iterrows():
    if pd.notna(row['bert_sentiment']):
        plt.scatter(row['Date'], row['Close'], color=colors.get(row['bert_sentiment'], 'blue'), s=50)

plt.title('AAPL Price and News Sentiment Overlay')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()

In [ ]:
5. Вывод и интерпретация

print("Анализ завершен. График показывает влияние тональности новостей на поведение цены акции.")